In [1]:
import tensorflow as tf

In [2]:
def conv(x, w, b, strides, padding="SAME", name = None):
    weight = tf.Variable(tf.truncated_normal(w, stddev=0.1), name="weights")
    bias = tf.Variable(tf.truncated_normal(b, stddev=0.1), name="bias")
    return tf.nn.relu(tf.nn.conv2d(x, weight, strides, padding='SAME') + bias, name=name)

In [3]:
def fc(x, w, b, name = None):
    weight = tf.Variable(tf.truncated_normal(w, stddev=0.1), name="weights")
    bias = tf.Variable(tf.truncated_normal(b, stddev=0.1), name="bias")
    return tf.add(tf.matmul(x, weight), bias)

In [4]:
 # input_size:(?, 224, 224, 3)
def model(input, dropout = 0.4):
    # Part1
    with tf.name_scope('Conv_7x7'):
        layer_1 = tf.nn.relu(conv(input, [7, 7, 3, 64], [64], [1, 2, 2, 1], name = 'conv_layer_1'))
        # size = (?, 112, 112, 64)
        
    with tf.name_scope('MaxPoolx1'):
        maxpool_1 = tf.nn.max_pool(layer_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='maxpool_1')
        # size = (?, 56, 56, 64)
        
    with tf.name_scope('LocalRespNorm_1'):
        LocalRespNorm_1 = tf.nn.local_response_normalization(maxpool_1, name='LocalRespNorm_1')
        # size = (?, 56, 56, 64)
    
    #Part2
    with tf.name_scope('conv1x1'):
        layer_2 = tf.nn.relu(conv(LocalRespNorm_1, [1, 1, 64, 64], [64], [1, 1, 1, 1], name='conv_layer_2'))
        # size = (?, 56, 56, 64)
    
    with tf.name_scope('conv3x3'):
        layer_3 = tf.nn.relu(conv(layer_2, [3, 3, 64, 192], [192], [1, 1, 1, 1], name='conv_layer_3'))
        # size = (?, 56, 56, 192)
    
    with tf.name_scope('LocalRespNorm_2'):
        LocalRespNorm_2 = tf.nn.local_response_normalization(layer_3, name='LocalRespNorm_2')
        # size = (?, 56, 56, 192)
    
    with tf.name_scope('MaxPool_2'):
        maxpool_2 = tf.nn.max_pool(LocalRespNorm_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', 
                                   name='maxpool_2')
        # size = (?, 28, 28, 192)
        
    #inception1
    with tf.name_scope('inception_1'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_1_1 = tf.nn.relu(conv(maxpool_2, [1, 1, 192, 64], [64], [1, 1, 1, 1], 
                                                name='inception_1_1'))
                # size = (?, 28, 28, 64)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_1_2_1 = tf.nn.relu(conv(maxpool_2, [1, 1, 192, 96], [96], [1, 1, 1, 1], 
                                                  name='inception_1_2_1'))
                # size = (?, 28, 28, 96)
            with tf.name_scope('conv3x3'):
                inception_1_2_2 = tf.nn.relu(conv(inception_1_2_1, [3, 3, 96, 128], [128], [1, 1, 1, 1], 
                                                  name='inception_1_2_2'))
                # size = (?, 28, 28, 128)
        
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_1_3_1 = tf.nn.relu(conv(maxpool_2, [1, 1, 192, 16], [16], [1, 1, 1, 1], 
                                                  name='inception_1_3_1'))
                 # size = (?, 28, 28, 16)
            
            with tf.name_scope('conv5x5'):
                inception_1_3_2 = tf.nn.relu(conv(inception_1_3_1, [5, 5, 16, 32], [32], [1, 1, 1, 1], 
                                                  name='inception_1_3_2'))
                # size = (?, 28, 28, 32)
            
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_1_4_1 = tf.nn.max_pool(maxpool_2, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_1_4_1')
                # size = (?, 28, 28, 192)
                
            with tf.name_scope('conv1x1'):
                inception_1_4_2 = tf.nn.relu(conv(inception_1_4_1, [1, 1, 192, 32], [32], [1, 1, 1, 1], 
                                                  name='inception_1_4_2'))
                # size = (?, 28, 28, 32)

        with tf.name_scope('DepthConcat'):
                inception_1 = tf.concat([inception_1_1, inception_1_2_2, inception_1_3_2, inception_1_4_2], 3)
                # size = (?, 28, 28, 256)
        
    #inception2
    with tf.name_scope('inception_2'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_2_1 = tf.nn.relu(conv(inception_1, [1, 1, 256, 128], [128], [1, 1, 1, 1], 
                                                name='inception_2_1'))
                # size = (?, 28, 28, 128)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_2_2_1 = tf.nn.relu(conv(inception_1, [1, 1, 256, 128], [128], [1, 1, 1, 1], 
                                                name='inception_2_2_1'))
                # size = (?, 28, 28, 128)
            with tf.name_scope('conv3x3'):
                inception_2_2_2 = tf.nn.relu(conv(inception_2_2_1, [3, 3, 128, 192], [192], [1, 1, 1, 1], 
                                                name='inception_2_2_2'))
                # size = (?, 28, 28, 192)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_2_3_1 = tf.nn.relu(conv(inception_1, [1, 1, 256, 32], [32], [1, 1, 1, 1], 
                                                name='inception_2_3_1'))
                # size = (?, 28, 28, 32)
            with tf.name_scope('conv5x5'):
                inception_2_3_2 = tf.nn.relu(conv(inception_2_3_1, [5, 5, 32, 96], [96], [1, 1, 1, 1], 
                                                name='inception_2_3_2'))
                # size = (?, 28, 28, 96)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_2_4_1 = tf.nn.max_pool(inception_1, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_2_4_1')
                # size = (?, 28, 28, 256)
            with tf.name_scope('conv1x1'):
                inception_2_4_2 = tf.nn.relu(conv(inception_2_4_1, [1, 1, 256, 64], [64], [1, 1, 1, 1], 
                                                name='inception_2_4_2'))
                # size = (?, 28, 28, 64)
                
        with tf.name_scope('DepthConcat'):
            inception_2 = tf.concat([inception_2_1, inception_2_2_2, inception_2_3_2, inception_2_4_2], 3)
            # size = (?, 28, 28, 480)
            
    # MaxPool_3
    with tf.name_scope('MaxPool_3'):
            maxpool_3 = tf.nn.max_pool(inception_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME",
                               name='maxpool_3')
            # size = (?, 14, 14, 480)
    
    #inception3
    with tf.name_scope('inception_3'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_3_1 = tf.nn.relu(conv(maxpool_3, [1, 1, 480, 192], [192], [1, 1, 1, 1], 
                                                name='inception_3_1'))
                # size = (?, 14, 14, 192)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_3_2_1 = tf.nn.relu(conv(maxpool_3, [1, 1, 480, 96], [96], [1, 1, 1, 1], 
                                                name='inception_3_2_1'))
                # size = (?, 14, 14, 96)
            with tf.name_scope('conv3x3'):
                inception_3_2_2 = tf.nn.relu(conv(inception_3_2_1, [3, 3, 96, 208], [208], [1, 1, 1, 1], 
                                                name='inception_3_2_2'))
                # size = (?, 14, 14, 208)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_3_3_1 = tf.nn.relu(conv(maxpool_3, [1, 1, 480, 16], [16], [1, 1, 1, 1], 
                                                name='inception_3_3_1'))
                # size = (?, 14, 14, 16)
            with tf.name_scope('conv5x5'):
                inception_3_3_2 = tf.nn.relu(conv(inception_3_3_1, [5, 5, 16, 48], [48], [1, 1, 1, 1], 
                                                name='inception_3_3_2'))
                # size = (?, 14, 14, 48)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_3_4_1 = tf.nn.max_pool(maxpool_3, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_3_4_1')
                # size = (?, 14, 14, 480)
            with tf.name_scope('conv1x1'):
                inception_3_4_2 = tf.nn.relu(conv(inception_3_4_1, [1, 1, 480, 64], [64], [1, 1, 1, 1], 
                                                name='inception_3_4_2'))
                # size = (?, 14, 14, 64)
                
        with tf.name_scope('DepthConcat'):
            inception_3 = tf.concat([inception_3_1, inception_3_2_2, inception_3_3_2, inception_3_4_2], 3)
            # size = (?, 14, 14, 512)
            
    # inception4
    with tf.name_scope('inception_4'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_4_1 = tf.nn.relu(conv(inception_3, [1, 1, 512, 160], [160], [1, 1, 1, 1], 
                                                name='inception_4_1'))
                # size = (?, 14, 14, 160)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_4_2_1 = tf.nn.relu(conv(inception_3, [1, 1, 512, 112], [112], [1, 1, 1, 1], 
                                                name='inception_4_2_1'))
                # size = (?, 14, 14, 112)
            with tf.name_scope('conv3x3'):
                inception_4_2_2 = tf.nn.relu(conv(inception_4_2_1, [3, 3, 112, 224], [224], [1, 1, 1, 1], 
                                                name='inception_4_2_2'))
                # size = (?, 14, 14, 224)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_4_3_1 = tf.nn.relu(conv(inception_3, [1, 1, 512, 24], [24], [1, 1, 1, 1], 
                                                name='inception_4_3_1'))
                # size = (?, 14, 14, 24)
            with tf.name_scope('conv5x5'):
                inception_4_3_2 = tf.nn.relu(conv(inception_4_3_1, [5, 5, 24, 64], [64], [1, 1, 1, 1], 
                                                name='inception_4_3_2'))
                # size = (?, 14, 14, 64)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_4_4_1 = tf.nn.max_pool(inception_3, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_4_4_1')
                # size = (?, 14, 14, 512)
            with tf.name_scope('conv1x1'):
                inception_4_4_2 = tf.nn.relu(conv(inception_4_4_1, [1, 1, 512, 64], [64], [1, 1, 1, 1], 
                                                name='inception_4_4_2'))
                # size = (?, 14, 14, 64)
                
        with tf.name_scope('DepthConcat'):
            inception_4 = tf.concat([inception_4_1, inception_4_2_2, inception_4_3_2, inception_4_4_2], 3)
            # size = (?, 14, 14, 512)
    
    # inception5
    with tf.name_scope('inception_5'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_5_1 = tf.nn.relu(conv(inception_4, [1, 1, 512, 128], [128], [1, 1, 1, 1], 
                                                name='inception_5_1'))
                # size = (?, 14, 14, 128)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_5_2_1 = tf.nn.relu(conv(inception_4, [1, 1, 512, 128], [128], [1, 1, 1, 1], 
                                                name='inception_5_2_1'))
                # size = (?, 14, 14, 128)
            with tf.name_scope('conv3x3'):
                inception_5_2_2 = tf.nn.relu(conv(inception_5_2_1, [3, 3, 128, 256], [256], [1, 1, 1, 1], 
                                                name='inception_5_2_2'))
                # size = (?, 14, 14, 256)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_5_3_1 = tf.nn.relu(conv(inception_4, [1, 1, 512, 24], [24], [1, 1, 1, 1], 
                                                name='inception_5_3_1'))
                # size = (?, 14, 14, 24)
            with tf.name_scope('conv5x5'):
                inception_5_3_2 = tf.nn.relu(conv(inception_5_3_1, [5, 5, 24, 64], [64], [1, 1, 1, 1], 
                                                name='inception_5_3_2'))
                # size = (?, 14, 14, 64)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_5_4_1 = tf.nn.max_pool(inception_4, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_5_4_1')
                # size = (?, 14, 14, 512)
            with tf.name_scope('conv1x1'):
                inception_5_4_2 = tf.nn.relu(conv(inception_5_4_1, [1, 1, 512, 64], [64], [1, 1, 1, 1], 
                                                name='inception_5_4_2'))
                # size = (?, 14, 14, 64)
                
        with tf.name_scope('DepthConcat'):
            inception_5 = tf.concat([inception_5_1, inception_5_2_2, inception_5_3_2, inception_5_4_2], 3)
            # size = (?, 14, 14, 512)
            
    # inception6
    with tf.name_scope('inception_6'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_6_1 = tf.nn.relu(conv(inception_5, [1, 1, 512, 112], [112], [1, 1, 1, 1], 
                                                name='inception_6_1'))
                # size = (?, 14, 14, 112)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_6_2_1 = tf.nn.relu(conv(inception_5, [1, 1, 512, 114], [114], [1, 1, 1, 1], 
                                                name='inception_6_2_1'))
                # size = (?, 14, 14, 114)
            with tf.name_scope('conv3x3'):
                inception_6_2_2 = tf.nn.relu(conv(inception_6_2_1, [3, 3, 114, 288], [288], [1, 1, 1, 1], 
                                                name='inception_6_2_2'))
                # size = (?, 14, 14, 288)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_6_3_1 = tf.nn.relu(conv(inception_5, [1, 1, 512, 32], [32], [1, 1, 1, 1], 
                                                name='inception_6_3_1'))
                # size = (?, 14, 14, 32)
            with tf.name_scope('conv5x5'):
                inception_6_3_2 = tf.nn.relu(conv(inception_6_3_1, [5, 5, 32, 64], [64], [1, 1, 1, 1], 
                                                name='inception_6_3_2'))
                # size = (?, 14, 14, 64)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_6_4_1 = tf.nn.max_pool(inception_5, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_6_4_1')
                # size = (?, 14, 14, 512)
            with tf.name_scope('conv1x1'):
                inception_6_4_2 = tf.nn.relu(conv(inception_6_4_1, [1, 1, 512, 64], [64], [1, 1, 1, 1], 
                                                name='inception_6_4_2'))
                # size = (?, 14, 14, 64)
                
        with tf.name_scope('DepthConcat'):
            inception_6 = tf.concat([inception_6_1, inception_6_2_2, inception_6_3_2, inception_6_4_2], 3)
            # size = (?, 14, 14, 528)
    
    # inception7
    with tf.name_scope('inception_7'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_7_1 = tf.nn.relu(conv(inception_6, [1, 1, 528, 256], [256], [1, 1, 1, 1], 
                                                name='inception_7_1'))
                # size = (?, 14, 14, 256)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_7_2_1 = tf.nn.relu(conv(inception_6, [1, 1, 528, 160], [160], [1, 1, 1, 1], 
                                                name='inception_7_2_1'))
                # size = (?, 14, 14, 160)
            with tf.name_scope('conv3x3'):
                inception_7_2_2 = tf.nn.relu(conv(inception_7_2_1, [3, 3, 160, 320], [320], [1, 1, 1, 1], 
                                                name='inception_7_2_2'))
                # size = (?, 14, 14, 320)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_7_3_1 = tf.nn.relu(conv(inception_6, [1, 1, 528, 32], [32], [1, 1, 1, 1], 
                                                name='inception_7_3_1'))
                # size = (?, 14, 14, 32)
            with tf.name_scope('conv5x5'):
                inception_7_3_2 = tf.nn.relu(conv(inception_7_3_1, [5, 5, 32, 128], [128], [1, 1, 1, 1], 
                                                name='inception_7_3_2'))
                # size = (?, 14, 14, 128)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_7_4_1 = tf.nn.max_pool(inception_6, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_7_4_1')
                # size = (?, 14, 14, 528)
            with tf.name_scope('conv1x1'):
                inception_7_4_2 = tf.nn.relu(conv(inception_7_4_1, [1, 1, 528, 128], [128], [1, 1, 1, 1], 
                                                name='inception_7_4_2'))
                # size = (?, 14, 14, 128)
                
        with tf.name_scope('DepthConcat'):
            inception_7 = tf.concat([inception_7_1, inception_7_2_2, inception_7_3_2, inception_7_4_2], 3)
            # size = (?, 14, 14, 832)
            
    # MaxPool_4
    with tf.name_scope('MaxPool_4'):
            maxpool_4 = tf.nn.max_pool(inception_7, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME",
                               name='maxpool_4')
            # size = (?, 7, 7, 832)
    
    # inception8
    with tf.name_scope('inception_8'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_8_1 = tf.nn.relu(conv(maxpool_4, [1, 1, 832, 256], [256], [1, 1, 1, 1], 
                                                name='inception_8_1'))
                # size = (?, 14, 14, 256)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_8_2_1 = tf.nn.relu(conv(maxpool_4, [1, 1, 832, 160], [160], [1, 1, 1, 1], 
                                                name='inception_8_2_1'))
                # size = (?, 14, 14, 160)
            with tf.name_scope('conv3x3'):
                inception_8_2_2 = tf.nn.relu(conv(inception_8_2_1, [3, 3, 160, 320], [320], [1, 1, 1, 1], 
                                                name='inception_8_2_2'))
                # size = (?, 14, 14, 320)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_8_3_1 = tf.nn.relu(conv(maxpool_4, [1, 1, 832, 32], [32], [1, 1, 1, 1], 
                                                name='inception_8_3_1'))
                # size = (?, 14, 14, 32)
            with tf.name_scope('conv5x5'):
                inception_8_3_2 = tf.nn.relu(conv(inception_8_3_1, [5, 5, 32, 128], [128], [1, 1, 1, 1], 
                                                name='inception_8_3_2'))
                # size = (?, 14, 14, 128)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_8_4_1 = tf.nn.max_pool(maxpool_4, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_8_4_1')
                # size = (?, 14, 14, 528)
            with tf.name_scope('conv1x1'):
                inception_8_4_2 = tf.nn.relu(conv(inception_8_4_1, [1, 1, 832, 128], [128], [1, 1, 1, 1], 
                                                name='inception_8_4_2'))
                # size = (?, 14, 14, 128)
                
        with tf.name_scope('DepthConcat'):
            inception_8 = tf.concat([inception_8_1, inception_8_2_2, inception_8_3_2, inception_8_4_2], 3)
            # size = (?, 14, 14, 832)
    
    # inception9
    with tf.name_scope('inception_9'):
        with tf.name_scope('Part1'):
            with tf.name_scope('conv1x1'):
                inception_9_1 = tf.nn.relu(conv(inception_8, [1, 1, 832, 384], [384], [1, 1, 1, 1], 
                                                name='inception_9_1'))
                # size = (?, 14, 14, 384)
        
        with tf.name_scope('Part2'):
            with tf.name_scope('conv1x1'):
                inception_9_2_1 = tf.nn.relu(conv(inception_8, [1, 1, 832, 192], [192], [1, 1, 1, 1], 
                                                name='inception_9_2_1'))
                # size = (?, 14, 14, 192)
            with tf.name_scope('conv3x3'):
                inception_9_2_2 = tf.nn.relu(conv(inception_9_2_1, [3, 3, 192, 384], [384], [1, 1, 1, 1], 
                                                name='inception_9_2_2'))
                # size = (?, 14, 14, 384)
                
        with tf.name_scope('Part3'):
            with tf.name_scope('conv1x1'):
                inception_9_3_1 = tf.nn.relu(conv(inception_8, [1, 1, 832, 48], [48], [1, 1, 1, 1], 
                                                name='inception_9_3_1'))
                # size = (?, 14, 14, 48)
            with tf.name_scope('conv5x5'):
                inception_9_3_2 = tf.nn.relu(conv(inception_9_3_1, [5, 5, 48, 128], [128], [1, 1, 1, 1], 
                                                name='inception_9_3_2'))
                # size = (?, 14, 14, 128)
        
        with tf.name_scope('Part4'):
            with tf.name_scope('maxpool3x3'):
                inception_9_4_1 = tf.nn.max_pool(inception_8, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding="SAME",
                                               name='maxpool_9_4_1')
                # size = (?, 14, 14, 832)
            with tf.name_scope('conv1x1'):
                inception_9_4_2 = tf.nn.relu(conv(inception_9_4_1, [1, 1, 832, 128], [128], [1, 1, 1, 1], 
                                                name='inception_9_4_2'))
                # size = (?, 14, 14, 128)
                
        with tf.name_scope('DepthConcat'):
            inception_9 = tf.concat([inception_9_1, inception_9_2_2, inception_9_3_2, inception_9_4_2], 3)
            # size = (?, 14, 14, 1024)
    
    #Softmax_0
    with tf.name_scope('Softmax_0'):
        with tf.name_scope('AveragePool'):
            avgpool_0 = tf.nn.avg_pool(inception_3, ksize=[1, 5, 5, 1], strides=[1, 3, 3, 1], padding="VALID",
                                          name='avgpool_0')
            # size = (4,4,512)
            
        with tf.name_scope('conv'):
            softmax_0_conv = tf.nn.relu(conv(avgpool_0, [1, 1, 512, 1024], [1024], [1, 1, 1, 1], 
                                                name='softmax_0_conv'))
            # size = (4,4,1024)
        
        with tf.name_scope('fc1'):
            softmax_0_line = tf.reshape(softmax_0_conv, [-1, 4 * 4 * 1024])
            softmax_0_fc1 = tf.nn.dropout(tf.nn.relu(fc(softmax_0_line, [4 * 4 * 1024, 1000], [1000],
                                                        name='softmax_0_fc1')),dropout)
            
        with tf.name_scope('fc2'):
            softmax_0_fc2 = tf.nn.dropout(tf.nn.relu(fc(softmax_0_fc1, [1000, 1000], [1000],
                                                        name='softmax_0_fc2')),dropout)
        
        with tf.name_scope('softmax0'):
            softmax_0_out = tf.nn.softmax(softmax_0_fc2, name='softmax_0_out')
          
        
    #Softmax_1
    with tf.name_scope('Softmax_1'):
        with tf.name_scope('AveragePool'):
            avgpool_1 = tf.nn.avg_pool(inception_6, ksize=[1, 5, 5, 1], strides=[1, 3, 3, 1], padding="VALID",
                                          name='avgpool_1')
            # size = (4,4,528)
            
        with tf.name_scope('conv'):
            softmax_1_conv = tf.nn.relu(conv(avgpool_1, [1, 1, 528, 1024], [1024], [1, 1, 1, 1], 
                                                name='softmax_1_conv'))
            # size = (4,4,1024)
        
        with tf.name_scope('fc1'):
            softmax_1_line = tf.reshape(softmax_1_conv, [-1, 4 * 4 * 1024])
            softmax_1_fc1 = tf.nn.dropout(tf.nn.relu(fc(softmax_1_line, [4 * 4 * 1024, 1000], [1000],
                                                        name='softmax_1_fc1')),dropout)
            
        with tf.name_scope('fc2'):
            softmax_1_fc2 = tf.nn.dropout(tf.nn.relu(fc(softmax_1_fc1, [1000, 1000], [1000],
                                                        name='softmax_1_fc2')),dropout)
        
        with tf.name_scope('softmax1'):
            softmax_1_out = tf.nn.softmax(softmax_1_fc2, name='softmax_1_out')
            
    
    #Softmax_2
    with tf.name_scope('Softmax_2'):
        with tf.name_scope('AveragePool'):
            avgpool_2 = tf.nn.avg_pool(inception_9, ksize=[1, 5, 5, 1], strides=[1, 3, 3, 1], padding="VALID",
                                          name='avgpool_2')
            # size = (1,1,1024)
        
        with tf.name_scope('fc1'):
            softmax_2_line = tf.reshape(avgpool_2, [-1, 1 * 1 * 1024])
            softmax_2_fc1 = tf.nn.dropout(tf.nn.relu(fc(softmax_2_line, [1 * 1 * 1024, 1000], [1000],
                                                        name='softmax_2_fc1')),dropout)
        
        with tf.name_scope('softmax2'):
            softmax_2_out = tf.nn.softmax(softmax_2_fc1, name='softmax_2_out')
    return softmax_0_out, softmax_1_out, softmax_2_out

In [5]:
#  tensorboard --logdir=G:\py\CNN\logs
#http://localhost:6006/
with tf.name_scope('Input_data'):
    X = tf.placeholder(tf.float32, [None, 224, 224, 3], name="Input")
y = model(X)
sess = tf.Session()
writer = tf.summary.FileWriter("./logs", sess.graph)
writer.close()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
